In [ ]:
import os
from time import time
import numpy as np
import sys
sys.path.append('../')

from utils.codification_cnn import FitnessCNNParallel
from utils.codification_grew import FitnessGrow, ChromosomeGrow, HyperParams, Merger
from utils.codification_grew import Inputs, MaxPooling, AvPooling, OperationBlock, CNNGrow, IdentityGrow
from utils.datamanager import DataManager
from GA.random_search import RandomSearcher


In [ ]:
# Chromosome parameters
ChromosomeGrow._max_initial_blocks = 6
Merger._projection_type = ['normal', 'extend'][1]

HyperParams._GROW_RATE_LIMITS = [1.5, 5.]
HyperParams._N_CELLS = [1, 2]
HyperParams._N_BLOCKS = [2]
HyperParams._STEM = [32, 45]

OperationBlock._change_op_prob = 0.15
OperationBlock._change_concat_prob = 0.15

CNNGrow.filters_mul_range = [0.1, 1.2]
CNNGrow.possible_activations = ['relu', 'elu', 'prelu']
CNNGrow.dropout_range = [0, 0.7]
CNNGrow.possible_k = [1, 3, 5]
    
data_folder = '../../datasets/MNIST_variations'
command = 'python3 ../train_gen.py'
verbose = 0

gpus = 1


In [ ]:
# dataset params:
data_folder = data_folder
classes = []

# random search algorithm params:
time_limit = 12
population_batch = 10
individual_example = ChromosomeGrow.random_individual() 
save=True

fitness_cnn = FitnessGrow()    
# Fitness params
epochs = 15
batch_size = 128
verbose = verbose
redu_plat = False
early_stop = 0
warm_up_epochs = 0
base_lr = 0.05
smooth = 0.1
cosine_dec = False
lr_find = False
precise_eps = 75

include_time = False
test_eps = 100
augment = False

datasets = ['fashion_mnist', 'MB', 'MBI', 'MRB', 'MRD', 'MRDBI']
datasets = ['MRDBI']

In [ ]:
experiments_folder = '../../experiments/RS0'
description = "Evaluation of Grow V2 codification using random search" 

experiments_folder = experiments_folder
os.makedirs(experiments_folder, exist_ok=True)
for dataset in datasets:
    print("\nEVOLVING IN DATASET %s ...\n" % dataset)
    exp_folder = os.path.join(experiments_folder, dataset)
    folder = os.path.join(exp_folder, 'RS')
    fitness_folder = exp_folder
    fitness_file = os.path.join(fitness_folder, 'fitness_example')   
    os.makedirs(folder, exist_ok=True)

    try:
        RS = RandomSearcher.load_genetic_algorithm(folder=folder)
        # Load data
        num_clases = 100 if dataset == 'cifar100' else 10
        dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder, num_clases=num_clases) #, max_examples=8000)
        data = dm.load_data()
        fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                       epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                       warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find,
                       precise_epochs=precise_eps, include_time=include_time, test_eps=test_eps, augment=augment)

        fitness_cnn.save(fitness_file)
    except:
        # Load data
        num_clases = 100 if dataset == 'cifar100' else 10
        dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder, num_clases=num_clases) #, max_examples=8000)
        data = dm.load_data()
        fitness_cnn.set_params(data=data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                       epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                       warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find,
                       precise_epochs=precise_eps, include_time=include_time, test_eps=test_eps,  augment=augment)

        fitness_cnn.save(fitness_file)

        del dm, data

        fitness = FitnessCNNParallel()
        fitness.set_params(chrom_files_folder=fitness_folder, fitness_file=fitness_file, max_gpus=gpus,
                       fp=32, main_line=command)
        RS = RandomSearcher(time_limit=time_limit,
                          population_batch=population_batch,
                          individual_example=individual_example,
                          fitness_evaluator=fitness, 
                          maximize=False,
                          save=True,
                          folder=folder)
        RS.println(description)


    ti_all = time()
    winner, best_fit, ranking = RS.evolve()
    print("Total elapsed time: %0.3f" % (time() - ti_all))